In [14]:
import pandas as pd
from geopy.distance import geodesic
from joblib import Parallel, delayed

In [13]:
light_pollution = pd.read_csv('light_pollution_2020.csv')
county_coords = pd.read_csv('GEOINFO2023.GEOINFO-Data.csv')

In [15]:
county_coords['INTPTLAT'] = pd.to_numeric(county_coords['INTPTLAT'], errors='coerce')
county_coords['INTPTLON'] = pd.to_numeric(county_coords['INTPTLON'], errors='coerce')
light_pollution['Latitude'] = pd.to_numeric(light_pollution['Latitude'], errors='coerce')
light_pollution['Longitude'] = pd.to_numeric(light_pollution['Longitude'], errors='coerce')


In [16]:
county_coords = county_coords.dropna(subset=['INTPTLAT', 'INTPTLON'])
light_pollution = light_pollution.dropna(subset=['Latitude', 'Longitude'])

In [18]:
def find_nearest_county(lat, lon, county_coords):
    distances = county_coords.apply(
        lambda county: geodesic((lat, lon), (county['INTPTLAT'], county['INTPTLON'])).km,
        axis=1
    )
    nearest_idx = distances.idxmin()
    return county_coords.loc[nearest_idx, 'NAME']

In [ ]:
light_pollution['Nearest_County'] = Parallel(n_jobs=-1)(  # n_jobs=-1 使用所有可用核心
    delayed(find_nearest_county)(row['Latitude'], row['Longitude'], county_coords)
    for _, row in light_pollution.iterrows()
)

In [9]:
_, indices = tree.query(pollution_points)


In [10]:
light_pollution['Nearest_County'] = county_coords.iloc[indices]['NAME'].values


In [11]:
county_light_pollution = light_pollution.groupby('Nearest_County')['Brightness'].sum().reset_index()


In [12]:
county_light_pollution.to_csv('county_light_pollution_2020.csv', index=False)